In [1]:
import requests
from pprint import pprint

In [2]:
HOST = 'https://<your-askcos-ip>'

# ASKCOS Web API

## Endpoints

API endpoints exist for the following services (described in more depth below):

|Endpoint|Description|
|:---|---:|
|/api/retro/|Single-step retrosynthetic prediction for a given target molecule|
|/api/context/|Context recommender for a reaction|
|/api/forward/|Reaction (product) prediction given reactants and context|
|/api/selectivity/|Multi-task site selectivity given reactant|
|/api/treebuilder/|Synthetic pathway tree builder|
|/api/template/|Retrosynthetic template lookup|
|/api/fast-filter/|Coarse-filter for reactions plausibility|
|/api/scscore/|Synthetic complexity score|
|/api/buyables/|Buyable price|
|/api/celery/|Query the status of the celery workers|

## Making requests

GET requests can be made passing in required and optional parameters (service-specific parameters are described below for each endpoint). In many cases, you will be providing SMILES strings, which often may not play well with HTTP. You should always escape your parameters. In the examples below, the `requests` python package allows you to provide the parameters as a dictionary of key-value pairs and takes care of the url escaping for you. If you create your own url queries, please remember to escape your SMILES strings.

The API now sits at a server that expects communication using HTTPS. By default, a randomly generated self-signed certificate is used, which is considered insecure (despite being much more secure than not using HTTPS at all). The `requests` Python package, by default, will not allow you to use HTTPS to communicate with a server that doesn't have a valid certificate, however this behavior can be overriden by passing `verify=False` to the get request, as shown in the examples below.

Internally, the code behind the API will be converting any SMILES string you provide to a canonicalized SMILES string. Therefore, the results returned by the API may include a slightly modified string than the result parameters you provided. Keep note of this if using SMILES string searching to parse through results.

In the examples below, a dictionary named `params` will be built to contain the parameters for each service. `requests.get` will be used, giving the API endpoint and the params dictionary. This returns a `Response` object, and the final results can be obtained using `Response.json()`. The `pprint` module is used to nicely format the JSON results.

For each `params` dictionary, required key-value pairs will be identified with `# required`, and the rest of the values can be considered optional, as the values used here are the default values used (unless otherwise stated). In these examples, the number of results being returned is being significantly decreased to make this document more readable.

## /api/retro/  
Given a `target` SMILES string, predict precursors. The optional settings adjust how precursors are generated, or filtered after generation.  
  
The response will be a json with the original request variables as well as `precursors`, a list of the suggested results. The actualsuggested precursor smiles strings are stored in `smiles` (molecules concatenated with '.'), and `smiles_split` (list of molecules).

In [3]:
params = {
    # required
    'target': 'CN(C)CCOC(c1ccccc1)c1ccccc1',
    
    # optional with defaults shown
    'template_prioritization': 'Relevance',
    'precursor_prioritization': 'RelevanceHeuristic',
    'mincount': 0,
    'num_templates': 100,
    'max_cum_prob': 0.995,
    'apply_fast_filter': True,
    'filter_threshold': 0.75,
    
    # modified for this example
    'num_results': 3 # default is 100
}
resp = requests.get(HOST+'/api/retro/', params=params, verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'precursors': [{'group_id': 0,
                 'mapped_smiles': '[1CH3][2N]([3CH3])[4CH2][5CH2][900Cl].[6OH][7CH]([8c]1[9cH][10cH][11cH][12cH][13cH]1)[14c]1[15cH][16cH][17cH][18cH][19cH]1',
                 'necessary_reagent': '',
                 'num_examples': 1223,
                 'plausibility': 0.998188316822052,
                 'rank': 1,
                 'reacting_atoms': [5, 6],
                 'score': -0.005976811431568982,
                 'smiles': 'CN(C)CCCl.OC(c1ccccc1)c1ccccc1',
                 'smiles_split': ['CN(C)CCCl', 'OC(c1ccccc1)c1ccccc1'],
                 'template_score': 0.33462658524513245,
                 'templates': ['59c5118c05581eb9f5753c93',
                               '59c5118c05581eb9f5753c9d']},
                {'group_id': 0,
                 'mapped_smiles': '[1CH3][2N]([3CH3])[4CH2][5CH2][6OH].[900OH][7CH]([8c]1[9cH][10cH][11cH][12cH][13cH]1)[14c]1[15cH][16cH][17cH][18cH][19cH]1',
                 'necessary_reagent': '',
            

## /api/context/
Given `reactants` and `products`, suggest reaction contexts (catalyst, reagents, solvents, and temperature). The response will have `contexts`, a list of each suggested context in order of recommendation. The maximum number of results that can be returned is 18.

In [4]:
params = {
    'reactants': 'CN(C)CCCl.OC(c1ccccc1)c1ccccc1', #required
    'products': 'CN(C)CCOC(c1ccccc1)c1ccccc1', #required
    
    'num_results': 5, # default is 10
    'return_scores': 'true' # default is false
}
resp = requests.get(HOST+'/api/context/', params=params, verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'contexts': [{'catalyst': '',
               'reagent': 'Cc1ccccc1.[H][N-][H].[Na+]',
               'score': 0.3389342725276947,
               'solvent': '',
               'temperature': 94.4798812866211},
              {'catalyst': '',
               'reagent': 'c1ccccc1.[H][N-][H].[Na+]',
               'score': 0.12604427337646484,
               'solvent': '',
               'temperature': 101.66519927978516},
              {'catalyst': '',
               'reagent': 'Cc1ccccc1C.[H][N-][H].[Na+]',
               'score': 0.10769660025835037,
               'solvent': '',
               'temperature': 124.40972900390625},
              {'catalyst': '',
               'reagent': '',
               'score': 0.0048656458966434,
               'solvent': 'Cc1ccccc1',
               'temperature': 109.34728240966797},
              {'catalyst': '',
               'reagent': '',
               'score': 0.004308403469622135,
               'solvent': 'c1ccccc1',
               'temperat

## /api/forward/
Given `reactants`, and optionally `reagents` and `solvent`, suggest probably products. The response will have `outcomes`, with `smiles` of the product, `prob` (probability) of this being the major product, and relative `score` for reach suggestion.

In [5]:
params = {
    'reactants': 'CN(C)CCCl.OC(c1ccccc1)c1ccccc1', #required
    
    # optional with defaults shown
    'reagents': '',
    'solvent': '',
    
    'num_results': 5 # default is 100
}
resp = requests.get(HOST+'/api/forward/?', params=params, verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'error': 'API request timed out (limit 30s)',
 'request': {'num_results': ['5'],
             'reactants': ['CN(C)CCCl.OC(c1ccccc1)c1ccccc1'],
             'reagents': [''],
             'solvent': ['']}}


## /api/selectivity/
Given the `smiles` of a reactant molecule, return the multi-task predictions for various C-H functionalization reactions with site directed probabilities. The results are a list of each task with the `atom_scores` (probability of functionalization at this atomic site) for each `task`.

In [6]:
params = {
    'smiles': 'Cc1ccccc1', # required
}
resp = requests.get(HOST+'/api/selectivity/', params=params, verify=False)
pprint(resp.json())

{'results': {'smiles': 'Cc1ccccc1',
             'task_scores': [{'atom_scores': [1.4901161193847656e-07,
                                              8.940696716308594e-08,
                                              0.3286764323711395,
                                              0.0002091825008392334,
                                              0.9734209775924683,
                                              0.0002091825008392334,
                                              0.3286762237548828],
                              'smiles': 'Cc1ccccc1',
                              'task': 'BrBr'},
                             {'atom_scores': [0.0,
                                              0.0,
                                              0.3997066617012024,
                                              1.1116266250610352e-05,
                                              0.9975868463516235,
                                              1.1116266250610352e-05,
              

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2.4437904357910156e-06,
                                              0.018024682998657227,
                                              1.755356788635254e-05,
                                              0.8774557113647461,
                                              1.755356788635254e-05,
                                              0.01802465319633484],
                              'smiles': 'Cc1ccccc1',
                              'task': 'Nc1ccc([N+](=O)[O-])cc1'},
                             {'atom_scores': [0.00387689471244812,
                                              0.06139153242111206,
                                              0.7994834780693054,
                                              0.11980855464935303,
                                              0.4275699853897095,
                                              0.11980855464935303,
                                              0.7994835376739502],
                              'smiles': 'Cc1ccccc1

                                              0.6218416690826416],
                              'smiles': 'Cc1ccccc1',
                              'task': 'O=c1onc(-c2ccccc2)o1'},
                             {'atom_scores': [1.7762184143066406e-05,
                                              0.0001277327537536621,
                                              0.7531365156173706,
                                              0.00155717134475708,
                                              0.9557313323020935,
                                              0.0015572309494018555,
                                              0.7531365156173706],
                              'smiles': 'Cc1ccccc1',
                              'task': 'CC(=O)c1ccc(Br)cc1'},
                             {'atom_scores': [0.009305417537689209,
                                              0.039221733808517456,
                                              0.8888797163963318,
                           

                             {'atom_scores': [0.000696331262588501,
                                              0.0014270544052124023,
                                              0.8975743055343628,
                                              0.02365851402282715,
                                              0.9338842630386353,
                                              0.023658543825149536,
                                              0.897574245929718],
                              'smiles': 'Cc1ccccc1',
                              'task': 'C[Sn](C)(C)Cl'},
                             {'atom_scores': [0.00021633505821228027,
                                              0.05524104833602905,
                                              0.5350524187088013,
                                              0.1688767671585083,
                                              0.12877732515335083,
                                              0.1688767671585083,
                   

## /api/treebuilder/
Given the `smiles` of a target molecule, and various optional settings, resolve possible synthetic pathways terminating according to the stopping criteria provided through settings. The results are structued as a nested dictionary, where the children of chemicals are reactions and the children of reactions are chemicals.

In [7]:
params = {
    'smiles': 'CN(C)CCOC(c1ccccc1)c1ccccc1', # required
    
    # optional with defaults shown
    'max_depth': 4,
    'max_branching': 25,
    'expansion_time': 60,
    'max_ppg': 10,
    'template_count': 100,
    'max_cum_prob': 0.995,
    'chemical_property_logic': 'none',
    'max_chemprop_c': 0,
    'max_chemprop_n': 0,
    'max_chemprop_o': 0,
    'max_chemprop_h': 0,
    'chemical_popularity_logic': 'none',
    'min_chempop_reactants': 5,
    'min_chempop_products': 5,
    'filter_threshold': 0.75,
    
    'return_first': 'true' # default is false
}
resp = requests.get(HOST+'/api/treebuilder/', params=params, verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'request': {'chemical_popularity_logic': ['none'],
             'chemical_property_logic': ['none'],
             'expansion_time': ['60'],
             'filter_threshold': ['0.75'],
             'max_branching': ['25'],
             'max_chemprop_c': ['0'],
             'max_chemprop_h': ['0'],
             'max_chemprop_n': ['0'],
             'max_chemprop_o': ['0'],
             'max_cum_prob': ['0.995'],
             'max_depth': ['4'],
             'max_ppg': ['10'],
             'min_chempop_products': ['5'],
             'min_chempop_reactants': ['5'],
             'return_first': ['true'],
             'smiles': ['CN(C)CCOC(c1ccccc1)c1ccccc1'],
             'template_count': ['100']},
 'trees': [{'as_product': 44,
            'as_reactant': 59,
            'children': [{'children': [{'as_product': 172,
                                        'as_reactant': 4651,
                                        'children': [],
                                        'id': 1,
          

## /api/template/
Given a template `id` (these are returned with `/api/retro/` precursors) look up template details, such as `reaction_smarts`, and `references` (Reaxys IDs).

In [8]:
params = {
    'id': '59c5300605581eb9f584df3d' # required
}
resp = requests.get(HOST+'/api/template/', params=params, verify=False)
pprint(resp.json())

{'request': {'id': ['59c5300605581eb9f584df3d']},
 'template': {'_id': '59c5300605581eb9f584df3d',
              'count': 13,
              'dimer_only': False,
              'intra_only': True,
              'necessary_reagent': '',
              'reaction_smarts': '[#7:5]-[C:4](=[O;D1;H0:6])-[c:3]:[c;H0;D3;+0:1](:[#7;a:2])-[N;H0;D3;+0:9](-[C:10])-[c:8]:[#7;a:7]>>Cl-[c;H0;D3;+0:1](:[#7;a:2]):[c:3]-[C:4](-[#7:5])=[O;D1;H0:6].[#7;a:7]:[c:8]-[NH;D2;+0:9]-[C:10]',
              'references': ['4544223',
                             '5028471',
                             '5028471',
                             '5028471',
                             '5028471',
                             '5029289',
                             '8592467',
                             '8593425',
                             '8593425',
                             '23062042',
                             '23062042',
                             '24072327',
                             '38773479']}}


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## /api/fast-filter/
Given `reactants` and `products`, return coarse-filter `score` which can be interpreted as a likelihood the reaction may be successful. The resulting score should not be assumed to be linearly proportional to likelihood, but instead should be used to identify bad suggestions with low scores.

In [9]:
params = {
    'reactants': 'CN(C)CCCl.OC(c1ccccc1)c1ccccc1', # required
    'products': 'CN(C)CCOC(c1ccccc1)c1ccccc1' # required
}
resp = requests.get(HOST+'/api/fast-filter/', params=params, verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'request': {'products': ['CN(C)CCOC(c1ccccc1)c1ccccc1'],
             'reactants': ['CN(C)CCCl.OC(c1ccccc1)c1ccccc1']},
 'score': 0.998188316822052}


## /api/scscore/
Given a `smiles` string of a molecule, return the Synthetic Complexity `score`.

In [10]:
params = {
    'smiles': 'OC(c1ccccc1)c1ccccc1' # required
}
resp = requests.get(HOST+'/api/scscore/', params=params, verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'request': {'smiles': ['OC(c1ccccc1)c1ccccc1']}, 'score': 1.5127570072299896}


## /api/buyables/search
Given a query, return buyables information. The query can be a SMILES string, or the source field for buyable documents. Optionally, the SMILES string can be canonicalized, or the query string could be a regular expression as follows: '.*{}.*'.format(SMILES) for mongoDB. This endpoint returns `buyables`, a list of documents from mongoDB with `ppg` (price per gram), `smiles`, and `source` (if it exists).

In [5]:
params = {
    'q': 'OC(c1ccccc1)c1ccccc1', # required
    
    'source': '',
    'regex': False,
    'limit': 100,
    'canonicalize': True
}
resp = requests.get(HOST+'/api/buyables/search/', params=params, verify=False)
pprint(resp.json())

{'buyables': [{'_id': '5dfa4fd06e1e8a027c02fbd9',
               'ppg': 1.0,
               'smiles': 'OC(c1ccccc1)c1ccccc1'}],
 'search': 'OC(c1ccccc1)c1ccccc1'}


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## /api/celery/
Query the status of celery workers on the server. For each queue, the `active` and `available` will sum to the total number of spawned celery workers, where the active workers represent the current number of workers able to complete tasks.

In [12]:
resp = requests.get(HOST+'/api/celery/', verify=False)
pprint(resp.json())

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'queues': [{'available': 2,
             'busy': 0,
             'name': 'Context Recommender Coordinator',
             'queue': 'cr_coordinator'},
            {'available': 2,
             'busy': 0,
             'name': 'Context Recommender Worker',
             'queue': 'cr_network_worker'},
            {'available': 1,
             'busy': 1,
             'name': 'Forward Predictor Scoring Coordinator',
             'queue': 'sc_coordinator'},
            {'available': 2,
             'busy': 0,
             'name': 'Forward Predictor Worker',
             'queue': 'ft_worker'},
            {'available': 5,
             'busy': 7,
             'name': 'One-Step/Tree Builder Retrosynthesis Worker',
             'queue': 'tb_c_worker'},
            {'available': 1,
             'busy': 0,
             'name': 'Site Selectivity Worker',
             'queue': 'sites_worker'},
            {'available': 2,
             'busy': 0,
             'name': 'Tree Builder Coordinator',
       